### Trabalho 3
#### Igor Loredo - 11275071
#### Natã Botelho - 11275105

In [ ]:
%pip install ta
from ta import add_all_ta_features
from ta.utils import dropna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28985 sha256=189245321df67bfecd1011ea41d9d43ba8c48f0f8abe017c0b3de55f3c3dfd0e
  Stored in directory: /root/.cache/pip/wheels/bc/2a/c2/a56e77d07edc16a1fa7fb012667e55cb0643cfa65996bddecc
Successfully built ta


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn import tree

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')

In [ ]:
#Obter dados online da base do Yahoo
def get_stock_data(ticker, start, end, interval):
  start = pd.to_datetime([start]).view(int)[0]//10**9 # convert to unix timestamp.
  end = pd.to_datetime([end]).view(int)[0]//10**9 # convert to unix timestamp.
  url = 'https://query1.finance.yahoo.com/v7/finance/download/' + ticker + '?period1=' + str(start) + '&period2=' + str(end) + '&interval=' + str(interval) + '&events=history'
  print(url)
  df = pd.read_csv(url)
  return df

Dados da petrobras, selic e Itau

In [ ]:
petr4 = get_stock_data('PETR4.SA', '2005-01-01', '2021-12-31', '1d')
petr4.set_index(keys = 'Date', inplace = True)

itub4 = get_stock_data('ITUB4.SA', '2005-01-01', '2021-12-31', '1d')
itub4.set_index(keys = 'Date', inplace = True)

selic = get_stock_data('SELIC.BK', '2005-01-01', '2021-12-31', '1d')
selic.set_index(keys = 'Date', inplace = True)


https://query1.finance.yahoo.com/v7/finance/download/PETR4.SA?period1=1104537600&period2=1640908800&interval=1d&events=history
https://query1.finance.yahoo.com/v7/finance/download/ITUB4.SA?period1=1104537600&period2=1640908800&interval=1d&events=history
https://query1.finance.yahoo.com/v7/finance/download/SELIC.BK?period1=1104537600&period2=1640908800&interval=1d&events=history


In [ ]:
def transformDados(dadosTransform):
  for index, row in dadosTransform.iterrows():
    dadosTransform.at[index, 'High'] = row['High']/row['Close'] * row['Adj Close']
    dadosTransform.at[index, 'Low'] = row['Low']/row['Close'] * row['Adj Close']
    dadosTransform.at[index, 'Close'] = row['Adj Close']

In [ ]:
#Selecionar quais dados serão utilizados com os algoritmos abaixo
dadosSelic = selic.copy()
transformDados(dadosSelic)

dadosPetro = petr4.copy()
transformDados(dadosPetro)

dadosItau = itub4.copy()
transformDados(dadosItau)

In [ ]:
def windowsButAndSell(dadosTransfrom):
  dadosTransfrom['Action']=-1000   # No action

  jan_inicio = 0
  jan_tam = 11 #Tamanho da janela (número de dias)
  jan_fim = jan_inicio + jan_tam

  total_dados = dadosTransfrom.shape[0]
  cont = 0

  while cont < (total_dados):
    min = dadosTransfrom['Close'].iloc[jan_inicio:jan_fim].min()
    max = dadosTransfrom['Close'].iloc[jan_inicio:jan_fim].max()
    for i in range(jan_inicio, jan_fim+1):
      if ((dadosTransfrom['Close'].iloc[i] == min)):
        dadosTransfrom.loc[dadosTransfrom.index[i],'Action'] = 0 # Hold
        dadosTransfrom.loc[dadosTransfrom.index[i+1],'Action'] = 1 #Buy
      elif ((dadosTransfrom['Close'].iloc[i] == max)):
        dadosTransfrom.loc[dadosTransfrom.index[i],'Action'] = 0 # Hold
        dadosTransfrom.loc[dadosTransfrom.index[i+1],'Action'] = -1 # Sell
      elif (dadosTransfrom['Action'].iloc[i] == -1000):
        dadosTransfrom.loc[dadosTransfrom.index[i],'Action'] = 0 #Hold
    jan_inicio = jan_fim + 1
    jan_fim = jan_inicio + jan_tam
    cont  = jan_fim

In [ ]:

windowsButAndSell(dadosSelic)
windowsButAndSell(dadosPetro)
windowsButAndSell(dadosItau)


In [ ]:
#Cálculo das bandas de Bollinger
def calcVolatility(dados):
  from ta.volatility import BollingerBands
  ind_bb = BollingerBands(close=dados['Close'], window=20, window_dev=2)
  dados['bb_hb'] = ind_bb.bollinger_hband()
  dados['bb_lb'] = ind_bb.bollinger_lband()
  dados['bb_mb'] = ind_bb.bollinger_mavg()
  dados['bb_hr'] = dados['bb_hb']/dados['Close']
  dados['bb_lr'] = dados['bb_lb']/dados['Close']
  dados['bb_mr'] = dados['bb_mb']/dados['Close']

In [ ]:
# Cálculo do indicador RSI
def calcRSI(dados):
  from ta.momentum import RSIIndicator
  ind_rsi = RSIIndicator(close=dados['Close'], window=2)
  dados['RSI'] = ind_rsi.rsi()/100.0

In [ ]:
# usamos o roc, ajustamente porque traz um oscilador de momentum
def calcROC(dados):
  from ta.momentum import ROCIndicator
  ind_roc = ROCIndicator(close=dados['Close'], window=2)
  dados['ROC'] = ind_roc.roc()/100.0

In [ ]:
calcVolatility(dadosSelic)
calcRSI(dadosSelic)
calcROC(dadosSelic)

calcVolatility(dadosPetro)
calcRSI(dadosPetro)
calcROC(dadosPetro)

calcVolatility(dadosItau)
calcRSI(dadosItau)
calcROC(dadosItau)

justicar a escola dos metodos

In [ ]:
class trainClass:
    def __init__(self):
        self.din = 0
        self.dout = 0
        self.train_in = 0
        self.train_out = 0
        self.test_in = 0
        self.test_out = 0

In [ ]:
def prepareData(dados): 
  # Exclui as primeiras linhas do data frame, uma vez que nem todos os índices estão disponíveis nessas datas.
  dados = dados.iloc[20:]
  train = trainClass()
  # Criação da matriz de dados de entrada e do vetor de dados de saída dos algoritmos de ML
  train.din = dados[['bb_hr', 'bb_lr', 'bb_mr', 'ROC', 'RSI']].reset_index().drop(['Date'], axis=1).to_numpy()
  train.dout = dados['Action'].reset_index().drop(['Date'], axis=1).to_numpy()

  # Separação dos dados em conjunto de treinamento e conjunto de teste/validação.
  train.train_in =   train.din[0:1000]
  train.train_out = train.dout[1:1001]

  train.test_in =   train.din[1000:2000]
  train.test_out = train.dout[1001:2001]

  print(train.din.shape)
  print(train.dout.shape)

  return train
  # createMatriz(dados, 'ROC')

  

In [ ]:
selicDadosTreino = prepareData(dadosSelic)
petroDadosTreino = prepareData(dadosPetro)
itauDadosTreino = prepareData(dadosItau)

(1246, 5)
(1246, 1)
(4204, 5)
(4204, 1)
(4204, 5)
(4204, 1)


In [ ]:
def trainData(dadosTreino, clf):
  # random forest conseguimos um resulto muito bom, junto com árvore de decisão
  # ja na MLP conseguimos um resultado bom

  # Treina o modelo com os dados de entrada/saída
  clfMLPC.fit(dadosTreino.train_in, dadosTreino.train_out.ravel())

  # Avaliação dos resultados do conjunto de dados de treinamento
  y_pred_train = clf.predict(dadosTreino.train_in)
  print("Accuracy train:",metrics.accuracy_score(dadosTreino.train_out, y_pred_train))

  # Avaliação dos resultados do conjunto de dados de teste
  y_pred_test = clf.predict(dadosTreino.test_in)
  print("Accuracy test:",metrics.accuracy_score(dadosTreino.test_out, y_pred_test))

  # Resultado dos algoritmos para todo o conjunto de dados.
  y_pred_all = clf.predict(dadosTreino.din)

In [ ]:
# Algoritmos de aprendizado de máquina. Utilizar apenas um deles em cada teste.

# Rede Neural MLP com parâmetros solver e activation alterados para melhor desempenho de aprendizado. 
clfMLPC = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(10, ), random_state=1, max_iter=100000, activation='tanh')

# Árvore de Decisão
clfDTC = DecisionTreeClassifier(random_state=1, max_depth=20)

# Random Forest
clfRFC = RandomForestClassifier(random_state=1, max_depth=10, n_estimators=3)

In [ ]:
trainData(selicDadosTreino, clfMLPC)
trainData(selicDadosTreino, clfDTC)
trainData(selicDadosTreino, clfRFC)

In [ ]:





  # inclusão da saída dos algoritmos de ML no Dataframe "dados_ml". É possível compara essa saída com a saída desejada (ideal).
  dados['ML-Action'] = y_pred_all
  
  clfDTC.fit(train_in, train_out.ravel())

  # Avaliação dos resultados do conjunto de dados de treinamento
  y_pred_train = clf.predict(train_in)
  print("Accuracy train:",metrics.accuracy_score(train_out, y_pred_train))

  # Avaliação dos resultados do conjunto de dados de teste
  y_pred_test = clf.predict(test_in)
  print("Accuracy test:",metrics.accuracy_score(test_out, y_pred_test))

  # Resultado dos algoritmos para todo o conjunto de dados.
  y_pred_all = clf.predict(din)

  # inclusão da saída dos algoritmos de ML no Dataframe "dados_ml". É possível compara essa saída com a saída desejada (ideal).
  dados['ML-Action'] = y_pred_all

  clfRFC.fit(train_in, train_out.ravel())

  # Avaliação dos resultados do conjunto de dados de treinamento
  y_pred_train = clf.predict(train_in)
  print("Accuracy train:",metrics.accuracy_score(train_out, y_pred_train))

  # Avaliação dos resultados do conjunto de dados de teste
  y_pred_test = clf.predict(test_in)
  print("Accuracy test:",metrics.accuracy_score(test_out, y_pred_test))

  # Resultado dos algoritmos para todo o conjunto de dados.
  y_pred_all = clf.predict(din)

  # inclusão da saída dos algoritmos de ML no Dataframe "dados_ml". É possível compara essa saída com a saída desejada (ideal).
  dados['ML-Action'] = y_pred_all

  

In [ ]:
dataTrain(dadosSelic)
dataTrain(dadosPetro)
dataTrain(dadosItau)

In [ ]:
def avaliaResult(clf, train_in, train_out, ):
  

In [ ]:
# Cálculo do saldo e de outras estatísticas dos trades
def saldoCalc(dados):
  dados['Saldo'] = 0
  saldo = 100
  comprado=0
  val_neg = 0
  trades = 0
  trades_pos = 0
  media_pos = 0
  media_neg = 0

  for i in range(0, dados.shape[0]):
    if (dados['ML-Action'].iloc[i] == 1 and comprado == 0):
      val_neg = dados['Close'].iloc[i]
      comprado = 1
    elif (dados['ML-Action'].iloc[i] == 1 and comprado == -1):
      res_trade = val_neg / dados['Close'].iloc[i] - 1
      if res_trade > 0:
        trades_pos = trades_pos + 1
        media_pos = media_pos + res_trade
      else:
        media_neg = media_neg + res_trade
      saldo = saldo * (1 + res_trade)
      trades = trades + 1
      comprado = 0
    elif (dados['ML-Action'].iloc[i] == -1 and comprado == 0):
      comprado = -1
      val_neg = dados['Close'].iloc[i]
    elif (dados['ML-Action'].iloc[i] == -1 and comprado == 1):    
      res_trade = dados['Close'].iloc[i]/val_neg - 1
      if res_trade > 0:
        trades_pos = trades_pos + 1
        media_pos = media_pos + res_trade
      else:
        media_neg = media_neg + res_trade
      saldo = saldo * (1 + res_trade)
      trades = trades + 1
      comprado = 0
    dados.loc[dados.index[i], 'Saldo'] = saldo

  if (trades_pos > 0):
    print("Trades total:", trades, "Trades positivos:", trades_pos, "(", round(trades_pos/trades*100, 2), "%)")
    print("Media trades positivos:", round(media_pos/trades_pos*100, 2), "%, Media trades negativos:", round(media_neg/(trades-trades_pos)*100, 2), "%")
    print("Rent. buy and hold:", round((dados['Close'].iloc[-1]/dados['Close'].iloc[0]-1)*100, 2), "%")
    print("Rent. ML trade:", round((dados['Saldo'].iloc[-1]/dados['Saldo'].iloc[0]-1)*100, 2), "%")
  else:
    print("Não foram feitos trades no período.")

In [ ]:
# Normalização e apresentação gráfica dos resultados
def graphPlot(dados):
  dados[['Close', 'Saldo']] = dados[['Close', 'Saldo']] / dados[['Close', 'Saldo']].iloc[1000] * 100
  dados[['Close', 'Saldo']].iloc[1000:].plot(figsize = (15,5), title='Curva de capital');
#dados[['ml-buy','Saldo']]

In [ ]:
print("Resultados MLPClassifier:\n")
avaliaResult(clfMLPC)
saldoCalc()
graphPlot()

In [ ]:
print("\nResultados Árvore de Decisão:\n")
avaliaResult(clfDTC)
saldoCalc()
graphPlot()

In [ ]:
print("\nResultados Random Forest:\n")
avaliaResult(clfRFC)
saldoCalc()
graphPlot()

In [ ]:
# Cálculo do saldo e de outras estatísticas dos trades
def statusTrades(dadosTrades):
  dadosTrades['Saldo'] = 0
  saldo = 100
  comprado=0
  val_neg = 0
  trades = 0
  trades_pos = 0
  media_pos = 0
  media_neg = 0

  for i in range(0, dadosTrades.shape[0]):
    if (dadosTrades['ML-Action'].iloc[i] == 1 and comprado == 0):
      val_neg = dadosTrades['Close'].iloc[i]
      comprado = 1
    elif (dadosTrades['ML-Action'].iloc[i] == 1 and comprado == -1):
      res_trade = val_neg / dadosTrades['Close'].iloc[i] - 1
      if res_trade > 0:
        trades_pos = trades_pos + 1
        media_pos = media_pos + res_trade
      else:
        media_neg = media_neg + res_trade
      dadosTrades = dadosTrades * (1 + res_trade)
      trades = trades + 1
      comprado = 0
    elif (dadosTrades['ML-Action'].iloc[i] == -1 and comprado == 0):
      comprado = -1
      val_neg = dadosTrades['Close'].iloc[i]
    elif (dadosTrades['ML-Action'].iloc[i] == -1 and comprado == 1):    
      res_trade = dadosTrades['Close'].iloc[i]/val_neg - 1
      if res_trade > 0:
        trades_pos = trades_pos + 1
        media_pos = media_pos + res_trade
      else:
        media_neg = media_neg + res_trade
      dadosTrades = dadosTrades * (1 + res_trade)
      trades = trades + 1
      comprado = 0
    dadosTrades.loc[dados.index[i], 'Saldo'] = dadosTrades

  if (trades_pos > 0):
    print("Trades total:", trades, "Trades positivos:", trades_pos, "(", round(trades_pos/trades*100, 2), "%)")
    print("Media trades positivos:", round(media_pos/trades_pos*100, 2), "%, Media trades negativos:", round(media_neg/(trades-trades_pos)*100, 2), "%")
    print("Rent. buy and hold:", round((dadosTrades['Close'].iloc[-1]/dadosTrades['Close'].iloc[0]-1)*100, 2), "%")
    print("Rent. ML trade:", round((dadosTrades['Saldo'].iloc[-1]/dadosTrades['Saldo'].iloc[0]-1)*100, 2), "%")
  else:
    print("Não foram feitos trades no período.")

In [ ]:
def statistics(clfIn, dadosTransform, trainIn ,trainOut, testIn, testOut):
  # Avaliação dos resultados do conjunto de dados de treinamento
  y_pred_train = clfIn.predict(trainIn)
  print("Accuracy train:",metrics.accuracy_score(trainOut, y_pred_train))

  # Avaliação dos resultados do conjunto de dados de teste
  y_pred_test = clfIn.predict(testIn)
  print("Accuracy test:",metrics.accuracy_score(testOut, y_pred_test))

  # Resultado dos algoritmos para todo o conjunto de dados.
  y_pred_all = clfIn.predict(din)
  return y_pred_test

In [ ]:
def createMatriz(dadosMatriz, parameterFisrt, parameterSecond):
  dadosMatriz = dadosMatriz.iloc[20:]

  # Criação da matriz de dados de entrada e do vetor de dados de saída dos algoritmos de ML
  dinTransform = dadosMatriz[['bb_hr', 'bb_lr', 'bb_mr', parameterFisrt, parameterSecond]].reset_index().drop(['Date'], axis=1).to_numpy()
  doutTransform  = dadosMatriz['Action'].reset_index().drop(['Date'], axis=1).to_numpy()

  # Separação dos dados em conjunto de treinamento e conjunto de teste/validação.
  train_in =   dinTransform[0:1000]
  train_out = doutTransform[1:1001]

  test_in =   dinTransform[1000:2000]
  test_out = doutTransform[1001:2001]

  print(dinTransform.shape)
  print(doutTransform.shape)
  return train_in, train_out, test_in, test_out

In [ ]:
from ta.momentum import ROCIndicator
ind_roc = ROCIndicator(close=dadosItau['Close'], window=2)
dadosPetro['ROC'] = ind_roc.roc()/100.0

In [ ]:
# Cálculo do indicador RSI

from ta.momentum import RSIIndicator
ind_rsi = RSIIndicator(close=dadosPetro['Close'], window=2)
dadosPetro['RSI'] = ind_rsi.rsi()/100.0